In [3]:
# Code uses Yahoo Finance to download SP500 components since Jan 2020

import bs4 as bs
import requests
import yfinance as yf
import datetime

resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2019,12,31)
data = yf.download(tickers, start=start, end=end)
#print(data)

[*********************100%***********************]  505 of 505 completed

4 Failed downloads:
- OTIS: Data doesn't exist for startDate = 946702800, endDate = 1577768400
- BRK.B: No data found, symbol may be delisted
- CARR: Data doesn't exist for startDate = 946702800, endDate = 1577768400
- BF.B: No data found for this date range, symbol may be delisted


In [1]:
#Copy data to .csv
#data.shape
#data.to_csv('sp500_returns.csv')

In [5]:
#Slice Open Prices
sp500_open=data.iloc[:, data.columns.get_level_values(0)=='Open']
sp500_open.head()
#Slice Adj. Close Prices
sp500_adjclose=data.iloc[:, data.columns.get_level_values(0)=='Adj Close']
sp500_adjclose.head()

Adj Close                                                       \
                    A AAL AAP      AAPL ABBV       ABC      ABMD       ABT   
Date                                                                         
2000-01-03  44.683800 NaN NaN  0.863657  NaN  2.968561  18.25000  6.539179   
2000-01-04  41.270466 NaN NaN  0.790842  NaN  2.765887  17.81250  6.352343   
2000-01-05  38.710434 NaN NaN  0.802415  NaN  2.980481  18.00000  6.340668   
2000-01-06  37.236496 NaN NaN  0.732975  NaN  3.206998  18.03125  6.562534   
2000-01-07  40.339535 NaN NaN  0.767695  NaN  3.624268  17.93750  6.632595   

                           ...                                                 \
           ACN       ADBE  ...       XLNX        XOM      XRAY        XRX XYL   
Date                       ...                                                  
2000-01-03 NaN  16.274673  ...  33.169758  20.730383  6.938441  51.060867 NaN   
2000-01-04 NaN  14.909401  ...  32.415897  20.333307  6.938441  48.685936 NaN   
2000-01-05 NaN  15.204173  ...  31.706371  21.441793  7.058388  51.192802 NaN   
2000-01-06 NaN  15.328290  ...  28.957006  22.550280  7.067613  50.137272 NaN   
2000-01-07 NaN  16.072983  ...  32.238522  22.484110  7.049161  51.324734 NaN   

                                                    
                 YUM ZBH       ZBRA       ZION ZTS  
Date                                                
2000-01-03  4.988526 NaN  25.027779  40.342747 NaN  
2000-01-04  4.888254 NaN  24.666666  38.389206 NaN  
2000-01-05  4.913321 NaN  25.138889  38.343781 NaN  
2000-01-06  4.871542 NaN  23.777779  38.888947 NaN  
2000-01-07  4.762912 NaN  23.513889  38.979809 NaN  

[5 rows x 505 columns]

In [6]:
# Calculate daily returns
sp500_returns=sp500_adjclose.pct_change()
sp500_returns.head()

Adj Close                                                           \
                   A AAL AAP      AAPL ABBV       ABC      ABMD       ABT ACN   
Date                                                                            
2000-01-03       NaN NaN NaN       NaN  NaN       NaN       NaN       NaN NaN   
2000-01-04 -0.076389 NaN NaN -0.084311  NaN -0.068273 -0.023973 -0.028572 NaN   
2000-01-05 -0.062031 NaN NaN  0.014634  NaN  0.077586  0.010526 -0.001838 NaN   
2000-01-06 -0.038076 NaN NaN -0.086539  NaN  0.076000  0.001736  0.034991 NaN   
2000-01-07  0.083333 NaN NaN  0.047368  NaN  0.130112 -0.005199  0.010676 NaN   

                      ...                                              \
                ADBE  ...      XLNX       XOM      XRAY       XRX XYL   
Date                  ...                                               
2000-01-03       NaN  ...       NaN       NaN       NaN       NaN NaN   
2000-01-04 -0.083889  ... -0.022727 -0.019154  0.000000 -0.046512 NaN   
2000-01-05  0.019771  ... -0.021888  0.054516  0.017287  0.051491 NaN   
2000-01-06  0.008163  ... -0.086713  0.051697  0.001307 -0.020619 NaN   
2000-01-07  0.048583  ...  0.113324 -0.002934 -0.002611  0.023684 NaN   

                                                  
                 YUM ZBH      ZBRA      ZION ZTS  
Date                                              
2000-01-03       NaN NaN       NaN       NaN NaN  
2000-01-04 -0.020101 NaN -0.014428 -0.048424 NaN  
2000-01-05  0.005128 NaN  0.019144 -0.001183 NaN  
2000-01-06 -0.008503 NaN -0.054144  0.014218 NaN  
2000-01-07 -0.022299 NaN -0.011098  0.002336 NaN  

[5 rows x 505 columns]

In [7]:
#manually calculate correlation coefficents - normalise by stdev.
import numpy as np

m = sp500_returns.mean(axis=0)
s = sp500_returns.std(ddof=1, axis=0)
 
# normalised time-series as an input for PCA
combo_pca = (sp500_returns - m)/s
combo_pca=combo_pca.fillna(0)
np.nan_to_num(combo_pca, copy=True, nan=0.0, posinf=None, neginf=None)
c = np.cov(combo_pca.values.T)     # covariance matrix   
co = np.corrcoef(combo_pca.values.T) #correlation matrix
co

C:\Users\amuni\anaconda3\envs\pyvizenv\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\amuni\anaconda3\envs\pyvizenv\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning:

invalid value encountered in true_divide



array([[1.        , 0.21805355, 0.21754512, ..., 0.32162253, 0.30995047,
        0.13177314],
       [0.21805355, 1.        , 0.25980212, ..., 0.21136626, 0.35427047,
        0.10835477],
       [0.21754512, 0.25980212, 1.        , ..., 0.23768146, 0.25640857,
        0.11249299],
       ...,
       [0.32162253, 0.21136626, 0.23768146, ..., 1.        , 0.26651431,
        0.16893298],
       [0.30995047, 0.35427047, 0.25640857, ..., 0.26651431, 1.        ,
        0.09280184],
       [0.13177314, 0.10835477, 0.11249299, ..., 0.16893298, 0.09280184,
        1.        ]])

In [14]:

'''

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


class PrincipalComponentAnalysis(object):
    '''
    allows for linear regression using principal components
    '''

    def __init__(self, returns_matrix, n_components = 0.95):
        self.returns_matrix = returns_matrix
        self.pca = PCA(n_components = n_components) # when n_component between 0 and 1, it returns number of principal components needed to explain percentage of variance.
        self.principal_components = self._get_pca_components()
        self.pca_linear_regresion_results = self._predict_returns_linear_regr_pca()
        self.predict_components_linear_1shift = self._predict_components_linear_1shift()
        
    def _get_pca_components(self):
        principal_components = self.pca.fit_transform(self.returns_matrix)
        return principal_components
    
    def plot_pca_cummulative_explained_variance(self):
        plt.grid(True)
        plt.title('PCAs Cumulative Explained Variance')
        plt.xlabel('Number of Principal Components')
        plt.ylabel('Explained Variance')
        plt.plot(np.cumsum(self.pca.explained_variance_ratio_))
        return plt.show()
    
    def _predict_returns_linear_regr_pca(self):
        train_x, test_x, train_y, test_y = train_test_split(self.principal_components, self.returns_matrix, test_size=1/5, random_state=0)
        linear_regr = LinearRegression()
        linear_regr.fit(train_x, train_y)
        predicted_returns = linear_regr.predict(test_x)
        results = {}
        results['predicted_returns'] = predicted_returns
        results['R2'] = linear_regr.score(test_x, test_y) # Returns the coefficient of determination R^2 of the prediction.
        results['coefficients'] = linear_regr.coef_
        results['intercept'] = linear_regr.intercept_
        results['mean_error'] = sum(abs(predicted_returns-test_y))/len(test_y)
        results['same_direction'] = sum(np.where(predicted_returns*test_y > 0, 1, 0))/len(test_y)
        same_direction = np.where(predicted_returns*test_y > 0, 1, 0)
        results['same_direction_percent_day'] = 100*same_direction.sum(axis=1)/same_direction.shape[1]
        return results
    
    def _predict_components_linear_1shift(self):
        x = np.delete(self.principal_components, -1, axis = 0) # deleting last day, lost when shifting to target next day
        y = np.delete(self.principal_components, 0, axis = 0) # deleting first day, to line up each day (x) with the following day
        train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=1/5, random_state=0)
        linear_regr = LinearRegression()
        linear_regr.fit(train_x, train_y)
        predicted_components = linear_regr.predict(test_x)
        results = {}
        results['predicted_components'] = predicted_components
        results['R2'] = linear_regr.score(test_x, test_y) # Returns the coefficient of determination R^2 of the prediction.
        results['coefficients'] = linear_regr.coef_
        results['intercept'] = linear_regr.intercept_
        results['mean_error'] = sum(abs(predicted_components-test_y))/len(test_y)
        return results 
 

--- 0.0 seconds ---


In [30]:
principal_components.shape

(505, 389)

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
n_components = .95
pca = PCA(n_components = n_components)
principal_components = pca.fit_transform(combo_pca.T)
train_x, test_x, train_y, test_y = train_test_split(principal_components, combo_pca.T, test_size=1/5, random_state=0)
linear_regr = LinearRegression()
linear_regr.fit(train_x, train_y)
predicted_returns = linear_regr.predict(test_x)
results = {}
results['predicted_returns'] = predicted_returns
results['R2'] = linear_regr.score(test_x, test_y) # Returns the coefficient of determination R^2 of the prediction.
results['coefficients'] = linear_regr.coef_
results['intercept'] = linear_regr.intercept_
#results['mean_error'] = sum(abs(predicted_returns-test_y))/len(test_y)
results['same_direction'] = sum(np.where(predicted_returns*test_y > 0, 1, 0))/len(test_y)
same_direction = np.where(predicted_returns*test_y > 0, 1, 0)
results['same_direction_percent_day'] = 100*same_direction.sum(axis=1)/same_direction.shape[1]

In [33]:
results

{'predicted_returns': array([[ 0.00000000e+00,  5.11986347e-01,  7.48878378e-01, ...,
         -2.30757473e-01, -1.00736400e+00, -6.63799199e-02],
        [ 0.00000000e+00, -1.13842698e+01, -7.12421472e+00, ...,
          5.15445236e+00,  3.09450797e+00, -3.41043208e-01],
        [ 0.00000000e+00,  1.89451714e+01, -9.89519334e+00, ...,
         -8.86963843e+00,  1.36750998e+00, -1.13723251e+00],
        ...,
        [ 0.00000000e+00, -3.81676532e+00, -8.68900820e-02, ...,
         -1.77360832e-01, -1.12170169e+00, -9.91380030e-02],
        [ 0.00000000e+00, -5.57881875e-01, -7.46920145e-01, ...,
         -4.34603118e+00,  3.22844392e+00,  1.15681137e-02],
        [ 0.00000000e+00,  4.05003557e+00,  1.07302539e+00, ...,
         -6.19034560e+00,  4.32241966e+00,  7.59024203e-01]]),
 'R2': -116.63333792166797,
 'coefficients': array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.02564191,  0.02810057,  0.02269234, ...,  0.042071

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


class PrincipalComponentAnalysis(object):
    '''
    allows for linear regression using principal components
    '''

    def __init__(self, returns_matrix, n_components = 0.95):
        self.returns_matrix = returns_matrix
        self.pca = PCA(n_components = n_components) # when n_component between 0 and 1, it returns number of principal components needed to explain percentage of variance.
        self.principal_components = self._get_pca_components()
        self.pca_linear_regresion_results = self._predict_returns_linear_regr_pca()
        self.predict_components_linear_1shift = self._predict_components_linear_1shift()
        
    def _get_pca_components(self):
        principal_components = self.pca.fit_transform(self.returns_matrix)
        return principal_components
    
    def plot_pca_cummulative_explained_variance(self):
        plt.grid(True)
        plt.title('PCAs Cumulative Explained Variance')
        plt.xlabel('Number of Principal Components')
        plt.ylabel('Explained Variance')
        plt.plot(np.cumsum(self.pca.explained_variance_ratio_))
        return plt.show()
    
    def _predict_returns_linear_regr_pca(self):
        train_x, test_x, train_y, test_y = train_test_split(self.principal_components, self.returns_matrix, test_size=1/5, random_state=0)
        linear_regr = LinearRegression()
        linear_regr.fit(train_x, train_y)
        predicted_returns = linear_regr.predict(test_x)
        results = {}
        results['predicted_returns'] = predicted_returns
        results['R2'] = linear_regr.score(test_x, test_y) # Returns the coefficient of determination R^2 of the prediction.
        results['coefficients'] = linear_regr.coef_
        results['intercept'] = linear_regr.intercept_
        results['mean_error'] = sum(abs(predicted_returns-test_y))/len(test_y)
        results['same_direction'] = sum(np.where(predicted_returns*test_y > 0, 1, 0))/len(test_y)
        same_direction = np.where(predicted_returns*test_y > 0, 1, 0)
        results['same_direction_percent_day'] = 100*same_direction.sum(axis=1)/same_direction.shape[1]
        return results
    
    def _predict_components_linear_1shift(self):
        x = np.delete(self.principal_components, -1, axis = 0) # deleting last day, lost when shifting to target next day
        y = np.delete(self.principal_components, 0, axis = 0) # deleting first day, to line up each day (x) with the following day
        train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=1/5, random_state=0)
        linear_regr = LinearRegression()
        linear_regr.fit(train_x, train_y)
        predicted_components = linear_regr.predict(test_x)
        results = {}
        results['predicted_components'] = predicted_components
        results['R2'] = linear_regr.score(test_x, test_y) # Returns the coefficient of determination R^2 of the prediction.
        results['coefficients'] = linear_regr.coef_
        results['intercept'] = linear_regr.intercept_
        results['mean_error'] = sum(abs(predicted_components-test_y))/len(test_y)
        return results 

In [9]:
# Sample test on any one ticker to evaluate how stocksplits etc. are correctly being picked up
aapl = yf.Ticker("AAPL")
hist = aapl.history(period="max")
aapl.splits
hist
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.10,0.10,0.10,0.10,469033600,0.0,0.0
1980-12-15,0.10,0.10,0.10,0.10,175884800,0.0,0.0
1980-12-16,0.09,0.09,0.09,0.09,105728000,0.0,0.0
1980-12-17,0.09,0.09,0.09,0.09,86441600,0.0,0.0
1980-12-18,0.09,0.09,0.09,0.09,73449600,0.0,0.0
